In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import torch
import generator
import plotly.graph_objects as go
from itertools import combinations
from scipy.spatial.transform import Rotation as R
from sklearn.decomposition import PCA

In [ ]:
def V(X, i, j):
    r_ij = X[i] - X[j]
    r = torch.norm(r_ij).pow(-6)
    return r.pow(2) - 2 * r

def center(X):
    Y = X - X[0]
    return Y
    
def system_potential(X):
    N, _ = X.shape
    energy = 0.0
    Y = center(X)
    for i in range(N - 1):
        for j in range(i + 1, N): 
            energy += V(Y, i, j)
    return energy

def line_search(X, energy, g, alpha_0=1.0, factor=0.5, c=1e-4, max_iter=10, tol=1e-8):
    alpha = alpha_0
    dot = torch.sum(-g * g).item()    
    for _ in range(max_iter):
        X_k = X - alpha * g
        energy_new = system_potential(X_k)

        if energy_new <= energy + c * alpha * dot:
            break
        alpha *= factor
        if alpha < tol:
            break
    return alpha

def gd_solve(natoms, lr=0.008, tol=1e-8, max_iter=2001, debug=True, debug_rate=400, save_initial=False, gen=generator.uniform_sphere):
    X = gen(natoms).clone().requires_grad_().to('cuda' if torch.cuda.is_available() else 'cpu', torch.float64)
    if save_initial:
        orig = X.clone()

    for _iter_ in range(max_iter):
        energy = system_potential(X) 
        g = torch.autograd.grad(energy, X)[0] 

        g_norm = g.norm().item()
        if g_norm < tol:
            if debug:
                print(f"Converged on step {_iter_}, energy: {energy.item():.6f}, gradient norm = {g_norm:.3e}")
            break
        
        with torch.no_grad(): 
            X -= line_search(X, energy, g) * g 
            X.requires_grad_() 

        if debug and _iter_ % debug_rate == 0:
            print(f"Step {_iter_}, energy: {energy.item():.6f}, gradient norm: {g_norm:.2e}")
    if save_initial:
        return orig.detach(), X.detach(), energy    
    return X.detach(), energy

In [ ]:
def plot_3d_points(points, energy):
    points_np = center(points).numpy()
    N,_ = points_np.shape
    x = points_np[:, 0]
    y = points_np[:, 1]
    z = points_np[:, 2]
    fig = go.Figure()
    for i, j in combinations(range(len(points_np)), 2):
        edge_len = np.linalg.norm(points_np[i] - points_np[j])
        V_ij = V(points, i, j)
        fig.add_trace(go.Scatter3d(
            x=[points_np[i, 0], points_np[j, 0]],
            y=[points_np[i, 1], points_np[j, 1]],
            z=[points_np[i, 2], points_np[j, 2]],
            mode='lines',
            line=dict(color='rgb(173, 216, 230, 0.4)', width=8),
            showlegend=False, 
            hovertext=f"Length: {edge_len:.2e}, Contributes {V_ij:.2e}J",
            hoverinfo="text",
        ))
    fig.add_trace(go.Scatter3d(
        x=x, y=y, z=z,
        mode='markers',
        hovertext=[f"Point {i + 1}" for i in range(N)],
        marker=dict(size=8, color='red'),
        showlegend=False
    ))
    fig.update_layout(
        scene=dict(
            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''),  # Remove grid, zero line, and ticks
            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''),
            zaxis=dict(showgrid=False, zeroline=False, showticklabels=False, title=''),
        ),
        title=f"{N}-Atom Configuration: {energy:.6f}J",
        width=600,
        height=400,
        plot_bgcolor='rgba(0,0,0,0)',
        paper_bgcolor='rgba(0,0,0,0)',
    )
    fig.update_scenes(xaxis_visible=False, yaxis_visible=False,zaxis_visible=False)
    fig.show()

In [ ]:
X_init, X, energy = gd_solve(3, debug=False, save_initial=True)
print(f"Final energy: {energy.item():.6f}J")
print(f"Final configuration:\n{X}")

In [20]:
for atoms in range(2,12):
    config = gd_solve(atoms, debug=True, debug_rate=2000)
    #plot_3d_points(*config)

Step 0, energy: -0.042197, gradient norm: 1.87e-01
Step 2000, energy: -1.000000, gradient norm: 1.47e-08
Step 0, energy: -0.221921, gradient norm: 7.62e-01
Step 2000, energy: -3.000000, gradient norm: 2.08e-07
Step 0, energy: -0.716374, gradient norm: 2.15e+00
Step 2000, energy: -6.000000, gradient norm: 4.78e-07
Step 0, energy: -1.726253, gradient norm: 4.39e+00
Step 2000, energy: -9.103852, gradient norm: 1.99e-07
Step 0, energy: -3.451955, gradient norm: 7.69e+00
Converged on step 66, energy: -12.712062, gradient norm = 7.254e-09
Step 0, energy: -6.024230, gradient norm: 1.16e+01
Converged on step 389, energy: -16.505384, gradient norm = 8.519e-09
Step 0, energy: -9.440679, gradient norm: 1.51e+01
Converged on step 502, energy: -19.765298, gradient norm = 4.281e-09
Step 0, energy: -13.541830, gradient norm: 1.67e+01
Converged on step 370, energy: -23.269812, gradient norm = 7.794e-09
Step 0, energy: -17.912500, gradient norm: 1.69e+01
Converged on step 706, energy: -26.771677, gradi

In [ ]:
# Verify that line search works right

X = generator.uniform_sphere(3).clone().requires_grad_(True)
for i in range(20):
    E = system_potential(X)
    g = torch.autograd.grad(E, X)[0]
    p = -g.view(-1)                        
    a = line_search(X, E, g, alpha_0=0.1)   # try 0.1 or whatever lr
    with torch.no_grad():
        X = X + a * p.view_as(X)
        X = X - X[0]
    X.requires_grad_(True)
    print(f"Iter {i:2d}: E = {E.item():.6f}, ‖grad‖ = {g.norm().item():.2e}, a = {a:.3f}")


In [ ]:
def bfgs_step(X, energy, g, H_inv, alpha=1.0, max_iter=10, tol=1e-8):
    N, _ = X.shape

    # flatten & slice out the free variables
    g_full = g.view(-1)
    g_free = g_full[3:]

    # descent dir in the free subspace
    s_free = - H_inv @ g_free

    # line search (still uses the full X and full g)
    alpha = line_search(X, energy, g, alpha_0=alpha, max_iter=max_iter, tol=tol)

    # take a step in the free variables
    x_full = X.clone().detach().view(-1)
    x_free = x_full[3:]
    x_free_new = x_free + alpha * s_free

    # reassemble X_new
    x_full[3:] = x_free_new
    X_new = x_full.view(N,3).requires_grad_(True)

    # eval new energy & gradient
    energy_new = system_potential(X_new)
    g_new_full = torch.autograd.grad(energy_new, X_new)[0]
    g_new = g_new_full.view(-1)
    g_new_free = g_new[3:]

    # BFGS inverse‐Hessian update on the free subspace
    s = x_free_new - x_free
    y = g_new_free - g_free
    ys = (y @ s).item()

    if ys <= 1e-8:
        H_inv_new = H_inv.clone()
    else:
        rho = 1.0 / ys
        I   = torch.eye(H_inv.shape[0], device=H_inv.device)
        V   = I - rho * torch.outer(s, y)
        with torch.no_grad():
            H_inv_new = V @ H_inv @ V.T + rho * torch.outer(s, s)

    return X_new, energy_new, g_new_full, H_inv_new, alpha

def bfgs(natoms, lr=0.008, g_tol=1e-8, energy_tol=1e-8, max_iter=2001, debug=True, debug_rate=400, save_initial=False, gen=generator.uniform_sphere):
    X = gen(natoms).clone().requires_grad_().to('cuda' if torch.cuda.is_available() else 'cpu', torch.float64)
    if save_initial:
        orig = X.clone()
    
    N, _ = X.shape
    H_inv = torch.eye(3 * (N - 1), dtype=torch.float64).to(X.device) 

    X = X.clone().detach().requires_grad_(True) # make sure autograd returns valid

    for _iter_ in range(max_iter):
        energy = system_potential(X)
        g = torch.autograd.grad(energy, X)[0]

        X_new, energy_new, g_new, H_inv, alpha = bfgs_step(X, energy, g, H_inv, alpha=lr)

        if abs(energy_new - energy) < energy_tol:
            if debug:
                print(f"Converged on step {_iter_}, energy: {energy_new.item():.6f}, gradient norm = {g_norm:.3e}")
            break

        g_norm = g.norm().item()
        if g_norm < g_tol:
            if debug:
                print(f"Converged on step {_iter_}, energy: {energy.item():.6f}, gradient norm = {g_norm:.3e}")
            break
        
        if debug and _iter_ % debug_rate == 0:
            print(f"Step {_iter_}, energy: {energy_new.item():.6f}, gradient norm: {g_norm:.2e}, step size: {alpha:.4e}")
        
        X, energy = X_new, energy_new

    if save_initial:
        return orig.detach(), X.detach(), energy_new 
    return X.detach(), energy_new

In [ ]:
config = bfgs(9, debug=True, g_tol=1e-8, energy_tol=1e-12, debug_rate=20)
plot_3d_points(*config)

In [17]:
for atoms in range(2,12):
    config = bfgs(atoms, debug=True, g_tol=1e-8, energy_tol=1e-8, debug_rate=400)
    #plot_3d_points(*config)

Step 0, energy: -0.042336, gradient norm: 1.87e-01, step size: 8.0000e-03
Converged on step 247, energy: -1.000000, gradient norm = 3.266e-03
Step 0, energy: -0.224873, gradient norm: 7.62e-01, step size: 8.0000e-03
Converged on step 113, energy: -3.000000, gradient norm = 2.256e-03
Step 0, energy: -0.743597, gradient norm: 2.15e+00, step size: 8.0000e-03
Converged on step 69, energy: -6.000000, gradient norm = 3.047e-03
Step 0, energy: -1.851902, gradient norm: 4.39e+00, step size: 8.0000e-03
Converged on step 59, energy: -9.103852, gradient norm = 9.032e-03
Step 0, energy: -3.862373, gradient norm: 7.69e+00, step size: 8.0000e-03
Converged on step 76, energy: -12.302927, gradient norm = 6.850e-03
Step 0, energy: -6.998799, gradient norm: 1.16e+01, step size: 8.0000e-03
Converged on step 94, energy: -16.505384, gradient norm = 4.265e-03
Step 0, energy: -11.062914, gradient norm: 1.51e+01, step size: 8.0000e-03
Converged on step 86, energy: -19.765297, gradient norm = 8.230e-03
Step 0,

In [24]:
config = bfgs(30, debug=True)
plot_3d_points(*config)


config = gd_solve(14, debug=True)
plot_3d_points(*config)

Step 0, energy: -0.000003, gradient norm: 4.91e+04, step size: 8.0000e-03
Converged on step 1, energy: -0.000003, gradient norm = 4.907e+04


Step 0, energy: -10.938299, gradient norm: 2.22e+02
Step 400, energy: -42.579276, gradient norm: 2.16e-07
Step 800, energy: -42.579276, gradient norm: 1.43e-06
Converged on step 925, energy: -42.579276, gradient norm = 7.502e-09
